### Example 4.2.1.1: 1-D interpolation on equally spaced input points

In [1]:
#Setting Peridynamic parameters
class PDoperator_:
    def __init__(self, number_input_points = None, xmin = None, xmax = None, 
               number_output_points = None, xvec_org = None, yvec_org = None, 
               n1order = None, nsize = None, order = None):
        self.totnode_org = number_input_points
        self.totnode = number_output_points
        self.xmin = xmin
        self.xmax = xmax
        self.xvec_org = xvec_org 
        self.yvec_org = yvec_org
        self.n1order = n1order
        self.nsize = nsize
        self.order = order
    

In [2]:
#Function that generates grid
class GridGenerator:
    def __init__(init, PDoperator):
        
        xmin = PDoperator.xmin 
        xmax = PDoperator.xmax
        totnode = PDoperator.totnode
        
        xvec = []
        dvolume = []
        dx = (xmax - xmin)/ totnode
        for i in range(totnode):
            xvec.append(round(xmin + dx*i +dx/2.0,6))
            dvolume.append(round(dx*1*1,6))
        init.xvec = xvec
        init.dvolume = dvolume
    
    

In [3]:
#Function that creates lump volumes
def LumpVolumes(PDoperator, gridgenerator):
    
    totnode_org = PDoperator.totnode_org 
    totnode = PDoperator.totnode 
    xvec_org = PDoperator.xvec_org
    
    xvec = gridgenerator.xvec 
    dvolume = gridgenerator.dvolume
    
    
    dvolumes_org = [0] * totnode_org
    tiny = 1E-7
    rho = [0]*totnode_org
    dvolume_org = [0]*totnode_org
    for i in range(totnode):
        rhosum = 0.0
        for j in range(totnode_org):
            rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
            rhosum = rhosum + rho[j]
        for j in range(totnode_org):
            dvolume_org[j] = dvolume_org[j] + (rho[j] / rhosum)*dvolume[i]
    rhosum = 0.0;
    for j in range(totnode_org):
        rhosum += dvolume_org[j]
    print("Total volume of output points         = ",totnode*dvolume[0] )
    print("Total volume of computed input points = ", rhosum)
    return dvolume_org

In [4]:
#Calculates size of Peridynamic operator
def getSize1D(n1order):
    nsize = n1order + 1
    return nsize

In [5]:
#Function that sets the order of the Differential Operators
def SetDiffOperators1D(nsize):
    order = list(range(0,nsize))
    return order

In [6]:
class NodeFamily:
    def __init__(self, xsi_order = None, xsivec = None, deltax = None):
        self.xsi_order = xsi_order
        self.xsivec = xsivec
        self.deltax = deltax

In [7]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
def GenerateNodeFamily(xvec_org, xvec):
    
    xsi_order = pd.DataFrame({'node':xvec_org})
    xsivec = pd.DataFrame({'node':xvec_org})
    #xsivec = pd.DataFrame()
    
    #Ordering nodes
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(math.sqrt((xvec[i]-xvec_org[j])**2))
        xsi_order[str(xvec[i])] = aux
        xsivec[str(xvec[i])] = aux
    
    xsi_order.set_index('node', inplace=True)
    for i in xsi_order:
        xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
    xsi_order_2 = pd.DataFrame({'node':xvec_org})
    for i in xsi_order:
        xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
    #Calculating deltas (horizon) we are selecting the largest bond
    deltax = []
    xsivec = xsivec.drop(columns="node")
    for i in xsivec:
        deltax.append(max(list(xsivec[str(i)])))
    
    NodeFam = NodeFamily()
    xsi_order_2 = xsi_order_2.drop(columns="node")
    NodeFam.xsi_order = xsi_order_2
    NodeFam.xsivec = xsivec
    NodeFam.deltax = deltax
    
    #aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]
    return NodeFam;

In [8]:
#Function that calculates the Peridynamics weights
def calc_weights(NodeFam, xvec):
    tiny = 0.000001
    weights = pd.DataFrame() 

    k=0    
    for i in xvec:
        xsivec_column = NodeFam.xsivec[str(i)].to_numpy()
        aux = []
        for xsi_mag in xsivec_column:
            aux.append((NodeFam.deltax[k]/(xsi_mag+tiny))**2)
        weights[str(i)] = aux
        k = k + 1
        
    return weights

In [9]:
import numpy as np
#Calculate p_operator
def p_operator_1d(x0,nsize,dvolumes_org, weights):
    
    x0_array = x0.to_numpy()
    weights_array = weights.to_numpy()
    aux2 = np.zeros((nsize,nsize))
    node_mem = 0;
   
    for i in x0_array:
        aux = np.zeros((nsize,1))
        for j in range(nsize):
            aux[j] = i**j
            
        #aux2 +=np.matmul(aux,np.transpose(aux))
        #aux2 = aux2*dvolumes_org[node_mem]
        aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]*weights[node_mem]
        #aux2 +=np.matmul(aux,np.transpose(aux))
        node_mem = node_mem + 1
    
    return aux2

In [10]:
#Form Diff A Matrix
import math

def FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam):
    #An A matrix must be created for each of the Peridynamic nodes
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    xsivec = pd.DataFrame()
    xsivec_mags = pd.DataFrame()
    
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(xvec_org[j]-xvec[i])
        xsivec[str(xvec[i])] = aux
        
    #weights must be calculated here it is going to be one for each Peridynamic node i.e. 200
    weights = calc_weights(NodeFam, xvec)

    A_prism = np.zeros((len(xvec), nsize,nsize ))
    j=0
    for i in xvec:     
        p_matrix = p_operator_1d(xsivec[str(i)],nsize, dvolumes_org,weights[str(i)])
        A_prism[j,:,:] = p_matrix
        j = j +1

    return A_prism
    

In [11]:
def FormInvDiffAmat1D(DiffAmat1D, nsize):
    a,b,c = DiffAmat1D.shape #get dimensions of DiffAmat1D
    DiffAmatInv1D = np.zeros((a, nsize,nsize ))
    for i in range(a)
        DiffAmatInv1D[i] = np.linalg.inv(DiffAmat1D[i])

    return DiffAmatInv1D

SyntaxError: invalid syntax (<ipython-input-11-3eecbc4e49db>, line 4)

In [ ]:
def FormDiffBvec1D(DiffAmatInv1D,nsize):
    a,b,c = DiffAmatInv1D.shape #get dimensions of DiffAmatInv1D
    DiffBvec1D_prism = np.zeros((a, nsize))
    DiffBvec1D = np.array([1,0,0])
    for i in range(a): 
        DiffBvec1D_prism[i,:] = np.matmul(DiffBvec1D,DiffAmatInv1D[i])
    return DiffBvec1D_prism

In [ ]:
import numpy as np
import math
#Function that calculates the 1D pass derivative operator
def Derivative_Operator_1D_PASS(PDoperator, gridgenerator, dvolumes_org):
    
    xvec = gridgenerator.xvec
    n1order = PDoperator.n1order
    PDoperator.nsize = getSize1D(n1order)
    PDoperator.order = SetDiffOperators1D(PDoperator.nsize)
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    DiffAmatInv1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmatInv 2D matrix
    DiffBvec1D = [0] * PDoperator.nsize
    DiffAvec1D = [0] * PDoperator.nsize
    plist = [0] * PDoperator.nsize
    blist = [0] * PDoperator.nsize
    weight = [0] * PDoperator.nsize
    rcvec = [0] * PDoperator.nsize
    wgt = [0] * PDoperator.totnode_org
    ndfam = [0] * PDoperator.totnode_org
    
   
    nsize = PDoperator.nsize
    
    xvec_org = PDoperator.xvec_org
    #Here is where we should calculate A matrix, inverse of A matrix and B vector; for every Peridynamic node.
    
    
    #Generate node family
    NodeFam = GenerateNodeFamily(xvec_org, xvec)
    #Form A array of A Matrices
    DiffAmat1D = FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam);    
    #Form Inverse of A Matrices
    DiffAmatInv1D = FormInvDiffAmat1D(DiffAmat1D, nsize)
    #Form B vector
    DiffBvec1D = FormDiffBvec1D(DiffAmatInv1D,nsize)
    #print("Interpolating for N=2")
    #for i in range(PDoperator.totnode):
        #GenerateNodeFamily_0
        
    
    return DiffBvec1D
    
    

In [ ]:
#The PDoperator class constructor has the following inputs
#PDoperator_name = PDoperator(number_input_points, xmin, xmax, output_points, 
#                             xvec_org, yvec_org)
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)

#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator, gridgenerator)
#Calculating the 1D derivative operator
h = Derivative_Operator_1D_PASS(PDoperator, gridgenerator,dvolumes_org)



In [ ]:
#np.zeros((len(gridgenerator.xvec), PDoperator.nsize))

In [ ]:
#h.to_excel("output.xlsx")

In [ ]:
print(h)
DiffBvec1D = np.array([1,0,0]) 
print(DiffBvec1D)
#print(np.matmul(DiffBvec1D,h[1]))

In [ ]:
print(gridgenerator.xvec)
print(xvec_org[2]-gridgenerator.xvec[0])

In [ ]:
print('Hello World')